In [1]:
#Setup
import os
import arcpy

In [2]:
#environment variables
arcpy.env.overwriteOutput = True

In [3]:
#load domains - used to check that all fields that have a domain have valid entries

#interest codes
tncInt = ['Fee Ownership', 'Conservation Easement', 'Deed Restrictions', 'Deed Restrictions - MonReq', 'Deed Restrictions - NoMon', 
'Management Lease or greement', 'Timber Lease or Agreement', 'Grazing Lease', 'Grazing Permit', 'Life Estate', 
'Right of Way Tract', 'Access Right of Way', 'Assist', 'Assist - Fee Ownership', 'Assist - Conservation Easement', 
'Assist - Deed Restriction', 'Transfer', 'Transfer - Fee Ownership', 'Transfer - Conservation Easement', 
'Transfer - Deed Restriction', 'Transfer - Life Estate', 'Transfer - Management Lease or Agreement', 'Transfer - Agreement']

#GAP codes
tnc_GAPCAT = {'1': 'managed for biodiversity – disturbance events proceed or are mimicked',
              '2': 'managed for biodiversity – disturbance events suppressed',
              '3': 'managed for multiple uses – subject to extractive (eg. mining or logging) or OHV use',
              '4': 'no known mandate for biodiversity protection'}

#and others as required

In [4]:
#get path to LRM report
lrmPath = 'D:/jplatt/projects/TNC_Lands/data/Fee_CE_Assessments/NAR Fee & Eas report 08-30-21.xlsx'

In [5]:
#set path to file geodatabase
gdbPath = 'D:/jplatt/projects/TNC_Lands/data/TNC_Lands_working.gdb'

In [6]:
#set path to TNC Lands data
tncLandsPath = os.path.join(gdbPath, 'TNC_Lands_Base')

In [7]:
#create table view on LRM report
if arcpy.Exists(os.path.join(gdbPath, 'lrmReport')):
    try:
        arcpy.Delete_management(os.path.join(gdbPath, 'lrmReport'))
    except arcpy.ExecuteError:
        print(arcpy.GetMessages(2))
    except Exception as e:
        print(e.args[0])

arcpy.ExcelToTable_conversion(lrmPath, os.path.join(gdbPath, 'lrmReport'), 'Sheet1')

<Result 'D:\\jplatt\\projects\\TNC_Lands\\data\\TNC_Lands_working.gdb\\lrmReport'>

In [8]:
indexes = arcpy.ListIndexes(os.path.join(gdbPath, 'lrmReport'))
try:
    if 'indx_trID' not in indexes: 
        arcpy.AddIndex_management(os.path.join(gdbPath, 'lrmReport'), ['LRM_Tract_ID'], 'indx_trID')
    else:
        arcpy.RemoveIndex_management(os.path.join(gdbPath, 'lrmReport'), ['LRM_Tract_ID'])
        arcpy.AddIndex_management(os.path.join(gdbPath, 'lrmReport'), ['LRM_Tract_ID'], 'indx_trID')
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(e.args[0])


In [26]:
#create search cursor on LRM report
state = 'SD'
tractIDs = [1000000,1000001,1000016,1000025,1000128,1000193]
for tractid in tractIDs:
    lrmCursor = arcpy.SearchCursor(os.path.join(gdbPath, 'lrmReport'), f"Primary_Geocode = '{state}' AND LRM_Tract_ID = {tractid}")

#arcpy.management.SelectLayerByAttribute("lrmReport", "NEW_SELECTION", "Primary_Geocode = 'SD'", None)

    for row in lrmCursor:
        tractID = row.getValue('LRM_Tract_ID')
        tractnm = row.getValue('Tract_Name')
        print(tractID, tractnm)


1000000 Institute of Range and the American Mustang (IRAM) 2
1000001 Wesleyan School
1000016 Hutton
1000025 South Dakota Office of School & Public Lands (fka South Dakota Parks & Wildlife Foundation)
1000128 Hauer 1
1000193 BHL Capital Corp. 1 (CalSD, LLC)


In [ ]:
#create search cursor on TNC Lands table

In [ ]:
#iterate over the LRM report
#select matching TNC Lands records
#start checking LRM fields